In [2]:
#importing Libraries 
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from IPython.display import display
import joblib
import numpy as np
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv("../Data/paper_train.csv")
df["nforest_type"] = df["nforest_type"].map({'MDF': 0, 'DDF':1, 'DEF': 2})

In [34]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split

# Get the counts of each class
class_counts = Counter(df['nforest_type'])

# Separate features and target
X = df.drop(columns=['id','nforest_type'])
# X = df.drop(columns=['nforest_type'])
y = df['nforest_type']

# Instantiate the RandomOverSampler
oversampler = RandomOverSampler(random_state=42)

# Oversample the minority classes
# X_oversampled, y_oversampled = oversampler.fit_resample(X, y)

# Split the oversampled data into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# Standardizing the features
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [35]:
# Create and train the CatBoostClassifier
model = CatBoostClassifier(
    iterations=500, learning_rate=0.1,
   loss_function='MultiClass', verbose=False)
model.fit(X_train, y_train)

In [29]:
# Make predictions on the val set
y_pred = model.predict(X_val)
 
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
report = classification_report(y_val, y_pred)
# printing metrics
print(f"Accuracy: {accuracy}")
print("Classification Report:\n", report)

Accuracy: 0.7159264931087289
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.78      0.73       571
           1       0.71      0.63      0.67       470
           2       0.80      0.73      0.76       265

    accuracy                           0.72      1306
   macro avg       0.73      0.71      0.72      1306
weighted avg       0.72      0.72      0.72      1306



# Submission

In [36]:
X.head()

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,...,BRI,BWDRVI,ARVI2,ARVI,Ashburn_Vegetation_Index,ARI,Alteration_Index,AFVI_2100,AFVI_1600,Adjusted_TSAVI
0,293,1927,1038,278,475,453,987,1773,2184,1900,...,0.284783,0.074472,0.503165,0.412451,0.614960,0.001092,3.937583,0.037452,0.293397,-0.150281
1,197,1598,697,201,347,228,682,1982,2449,2254,...,0.487988,0.083625,0.796732,0.686706,0.816277,0.001416,4.187956,0.263939,0.527618,-0.381403
2,929,1975,1031,982,1020,856,1220,2051,2421,2392,...,0.254051,0.041790,0.532351,0.326779,0.472906,0.000161,1.501499,0.448565,0.397604,-0.229597
3,132,1560,689,189,408,175,609,2117,2907,3024,...,0.553191,0.088235,0.898901,0.803536,0.890591,0.000809,3.767169,0.321115,0.628872,-0.633802
4,241,1944,1131,362,538,487,918,1549,1844,1702,...,0.255776,0.064922,0.471046,0.358789,0.555048,0.000769,3.416667,0.039978,0.201553,-0.182600


In [37]:
test = pd.read_csv("../Data/new_test.csv")
test = test[X.columns]
test.head()

,b1,b11,b12,b2,b3,b4,b5,b6,b7,b8,...,BRI,BWDRVI,ARVI2,ARVI,Ashburn_Vegetation_Index,ARI,Alteration_Index,AFVI_2100,AFVI_1600,Adjusted_TSAVI
0,69,1425,693,312,524,376,847,1821,2356,2378,...,0.365067,0.076803,0.687722,0.563045,0.726943,0.000728,2.533493,0.347838,0.548681,-0.326971
1,242,1514,691,343,522,324,718,1730,2178,2472,...,0.413399,0.075631,0.780338,0.625758,0.768240,0.000523,2.549133,0.431683,0.563073,-0.416220
2,218,2354,1118,292,596,410,965,2586,3226,3371,...,0.456491,0.084057,0.729161,0.634174,0.783126,0.000642,3.909910,0.197311,0.501894,-0.353045
3,350,2013,1134,306,572,475,982,1754,1935,2275,...,0.282164,0.076288,0.558753,0.461909,0.654545,0.000730,3.584282,0.079545,0.334702,-0.236195
4,185,1450,712,293,440,384,673,1487,1965,2213,...,0.376852,0.076616,0.646577,0.530849,0.704274,0.000787,2.949523,0.287359,0.513162,-0.205318


In [38]:

scaler.fit(test)
test_data = scaler.transform(test)
pred = model.predict(test_data)
pred = [item for sublist in pred for item in sublist]
pred[:3]

[2, 0, 0]